In [14]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your API KEY")

from langchain_openai import ChatOpenAI, OpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key=os.environ.get("OPENAI_API_KEY"))

In [21]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [30]:
import os


import os

# Specify the absolute path to the target file
target_file_path = r"\Users\mava08\OneDrive - AF Gruppen ASA\Datamateriale-endringer\CCB NS 8415 Underentreprise, avklarende kontraktsm_te, referat.pades.pdf"

# Get the current working directory
current_directory = os.getcwd()

# Find the relative path from the current directory to the target file
relative_path = os.path.relpath(target_file_path, start=current_directory)

print(f"The relative path to the file is: {relative_path}")

The relative path to the file is: ..\..\..\..\OneDrive - AF Gruppen ASA\Datamateriale-endringer\CCB NS 8415 Underentreprise, avklarende kontraktsm_te, referat.pades.pdf


In [31]:
loader = PyPDFLoader(relative_path)
pages = loader.load_and_split()

print(pages[0])

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(pages)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())



page_content='Identifikasjon  Rev:  Parefere : Side  \nCCB NS 8415 Underentreprise, avklarende kontraktsmøte, referat  07/28.02.2020  1 av 17 \n \n \n \nMØTEREFERAT  \n \nMøte type:            Avklarende kontraktsmøte  NS 8415   \nMøte nr. : 01/01 \n \nMøtedato : 02.10.2023   \nSted:  Ulvenveien 91 \n \nMøteleder:  Ola Randby   \nReferent: Ola Randby  \n \nProsjekt /nr.:  Construction City / 14100   \n \n \nDeltakere:    \nNavn   \nFirma / Funksjon   \nAnm .: \n \n Ola Randby  CCB  – Prosjektleder drift fase 2   \n Petter Tufte  \n \nRoger H årklau  \n CCB – Anleggsleder  plan  2-11 \n  \nCCB – Anleggslede r plan U2 -1  \n Haakon N. Kristiansen  \n \nStig Mathisen  \n \nHans Petter Berg  \n CCB – Prosjektingeniør  \n \nOBI – Markedssjef  \n \nOBI – Prosjektleder   \n    \n(X) = Ikke tilstede  \n \nReferat til:  \nDeltagere og stedfortredere.   \nKopi til:  \nCCB  \n \n \nAvklaring med f oretak:  \n \nFag / leveranse:  \n Himling og Systemvegg er \nForetak s \nnavn/nummer  Oslo B ygginn

In [38]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("Hva ble avtalt om avfallsmengde i avklarende kontraktsmøte 02.10.2023?")

'Det ble avtalt at alt avfall skal sorteres i henhold til prosjektets avfallsplan og kastes i riktig container. Prosjektet har et mål om en avfallsmengde på 18 kg/m2, og det er viktig at riktige tiltak treffes for å oppnå dette. Det ble også nevnt at det er mulig å organisere en returordning for himlingsplater fra Rockfon, men dette vil ha en kostnad som byggherren må dekke.'